# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import GridSearchCV
import pickle

DATABASE_FILENAME = '../db.sqlite3'
TABLE_NAME = 'disaster_message'
MODEL_PICKLE_FILENAME = 'trained_classifier.pkl'

nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\simon\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\simon\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\simon\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///' + DATABASE_FILENAME)
df = pd.read_sql_table(TABLE_NAME, engine)
X = df['message']
Y = df.iloc[:, 4:]
category_names = list(df.columns[4:])

In [3]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
X

0        Weather update - a cold front from Cuba that c...
1                  Is the Hurricane over or is it not over
2                          Looking for someone but no name
3        UN reports Leogane 80-90 destroyed. Only Hospi...
4        says: west side of Haiti, rest of the country ...
                               ...                        
26211    The training demonstrated how to enhance micro...
26212    A suitable candidate has been selected and OCH...
26213    Proshika, operating in Cox's Bazar municipalit...
26214    Some 2,000 women protesting against the conduc...
26215    A radical shift in thinking came about as a re...
Name: message, Length: 26216, dtype: object

In [5]:
Y

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26211,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26212,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26213,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26214,1,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, 'urlplaceholder')

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
# Pipiline simple version
simple_pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize))
    , ('tfidf', TfidfTransformer())
    , ('clf', MultiOutputClassifier(RandomForestClassifier()))])

simple_pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__ccp_alpha', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__max_samples', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf_

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

In [9]:
%%time
simple_pipeline.fit(X_train, Y_train)

Wall time: 9min 52s


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x00000271FB10BAF0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
Y_pred = simple_pipeline.predict(X_test)
    
# Calculate the accuracy for each of them.
for i in range(len(category_names)):
    print('Category {}: {} '.format(i, category_names[i]))
    print(classification_report(Y_test.iloc[:, i].values, Y_pred[:, i]))
    print('Accuracy {}\n\n'.format(accuracy_score(Y_test.iloc[:, i].values, Y_pred[:, i])))

Category 0: related 
              precision    recall  f1-score   support

           0       0.77      0.29      0.42      1512
           1       0.81      0.97      0.89      4997
           2       0.59      0.29      0.39        45

    accuracy                           0.81      6554
   macro avg       0.73      0.52      0.56      6554
weighted avg       0.80      0.81      0.77      6554

Accuracy 0.8097345132743363


Category 1: request 
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      5467
           1       0.89      0.45      0.59      1087

    accuracy                           0.90      6554
   macro avg       0.89      0.72      0.77      6554
weighted avg       0.90      0.90      0.88      6554

Accuracy 0.8988404028074458


Category 2: offer 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6522
           1       0.00      0.00      0.00        32

    acc

c:\users\simon\google drive\projects\udacity\data_scientist_nanodegree\core_curriculum\term_2\data_engineering\disaster_response_pipeline_project\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\simon\google drive\projects\udacity\data_scientist_nanodegree\core_curriculum\term_2\data_engineering\disaster_response_pipeline_project\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\simon\google drive\projects\udacity\data_scientist_nanodegree\core_curriculum\term_2\data_engineering\disaster_response


Accuracy 0.9661275556911809


Category 9: child_alone 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6554

    accuracy                           1.00      6554
   macro avg       1.00      1.00      1.00      6554
weighted avg       1.00      1.00      1.00      6554

Accuracy 1.0


Category 10: water 
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6122
           1       0.90      0.22      0.36       432

    accuracy                           0.95      6554
   macro avg       0.92      0.61      0.67      6554
weighted avg       0.94      0.95      0.93      6554

Accuracy 0.9472078120231919


Category 11: food 
              precision    recall  f1-score   support

           0       0.93      0.99      0.96      5813
           1       0.88      0.39      0.54       741

    accuracy                           0.92      6554
   macro avg       0.91      0.69      0.7

c:\users\simon\google drive\projects\udacity\data_scientist_nanodegree\core_curriculum\term_2\data_engineering\disaster_response_pipeline_project\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\simon\google drive\projects\udacity\data_scientist_nanodegree\core_curriculum\term_2\data_engineering\disaster_response_pipeline_project\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\simon\google drive\projects\udacity\data_scientist_nanodegree\core_curriculum\term_2\data_engineering\disaster_response


Category 17: death 
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6237
           1       0.86      0.09      0.17       317

    accuracy                           0.96      6554
   macro avg       0.91      0.55      0.57      6554
weighted avg       0.95      0.96      0.94      6554

Accuracy 0.9554470552334452


Category 18: other_aid 
              precision    recall  f1-score   support

           0       0.87      1.00      0.93      5701
           1       0.58      0.01      0.03       853

    accuracy                           0.87      6554
   macro avg       0.73      0.51      0.48      6554
weighted avg       0.83      0.87      0.81      6554

Accuracy 0.8703082087274947


Category 19: infrastructure_related 
              precision    recall  f1-score   support

           0       0.93      1.00      0.97      6120
           1       0.17      0.00      0.00       434

    accuracy                           0

c:\users\simon\google drive\projects\udacity\data_scientist_nanodegree\core_curriculum\term_2\data_engineering\disaster_response_pipeline_project\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\simon\google drive\projects\udacity\data_scientist_nanodegree\core_curriculum\term_2\data_engineering\disaster_response_pipeline_project\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\simon\google drive\projects\udacity\data_scientist_nanodegree\core_curriculum\term_2\data_engineering\disaster_response

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6526
           1       0.00      0.00      0.00        28

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      1.00      0.99      6554

Accuracy 0.9957277998169057


Category 26: aid_centers 
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6468
           1       0.00      0.00      0.00        86

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.97      0.99      0.98      6554

Accuracy 0.9868782422947818


Category 27: other_infrastructure 
              precision    recall  f1-score   support

           0       0.95      1.00      0.98      6249
           1       0.17      0.00      0.01       305

    accuracy                           0.95      6554
   macr

c:\users\simon\google drive\projects\udacity\data_scientist_nanodegree\core_curriculum\term_2\data_engineering\disaster_response_pipeline_project\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Let's print it in a more readable way:

In [11]:
print(classification_report(Y_test.iloc[:, 1:].values, np.array([x[1:] for x in Y_pred])))#, target_names = category_names))

              precision    recall  f1-score   support

           0       0.89      0.45      0.59      1087
           1       0.00      0.00      0.00        32
           2       0.80      0.60      0.69      2717
           3       0.61      0.05      0.10       513
           4       0.68      0.06      0.11       329
           5       1.00      0.06      0.11       186
           6       1.00      0.01      0.02       126
           7       0.75      0.03      0.05       226
           8       0.00      0.00      0.00         0
           9       0.90      0.22      0.36       432
          10       0.88      0.39      0.54       741
          11       0.91      0.22      0.36       556
          12       0.67      0.06      0.11        99
          13       0.67      0.01      0.03       149
          14       0.00      0.00      0.00        61
          15       0.50      0.01      0.02       220
          16       0.86      0.09      0.17       317
          17       0.58    

c:\users\simon\google drive\projects\udacity\data_scientist_nanodegree\core_curriculum\term_2\data_engineering\disaster_response_pipeline_project\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\simon\google drive\projects\udacity\data_scientist_nanodegree\core_curriculum\term_2\data_engineering\disaster_response_pipeline_project\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\simon\google drive\projects\udacity\data_scientist_nanodegree\core_curriculum\term_2\data_engineering\disaster_response_pipelin

### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
%%time
parameters = {
    'vect__ngram_range': ((1, 1), (1, 2))
    , 'vect__max_df': (0.5, 0.75, 1.0)
    , 'tfidf__use_idf': (True, False)
    #, 'clf__estimator__n_estimators': [50, 100, 200]
    , 'clf__estimator__min_samples_split': [2, 3, 4]
}

simple_pipeline_cv = GridSearchCV(simple_pipeline, param_grid = parameters)

Wall time: 0 ns


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
%%time
simple_pipeline_cv.fit(X_train, Y_train)
print('Best Parameters:', simple_pipeline_cv.best_params_)

In [ ]:
Y_pred = simple_pipeline_cv.predict(X_test)
    
print(classification_report(Y_test.iloc[:, 1:].values, np.array([x[1:] for x in Y_pred]), target_names = category_names))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [ ]:
pipeline = Pipeline([
    ('features', FeatureUnion([
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize))
            , ('tfidf', TfidfTransformer())]))
        , ('starting_verb', StartingVerbExtractor())]))
    , ('clf', RandomForestClassifier())])

pipeline.get_params()

In [ ]:
%%time
pipeline.fit(X_train, Y_train)

In [ ]:
Y_pred = pipeline.predict(X_test)
print(classification_report(Y_test.iloc[:, 1:].values, np.array([x[1:] for x in Y_pred]), target_names = category_names)) 

In [ ]:
%%time
parameters = {
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
        'features__text_pipeline__vect__max_features': (None, 5000, 10000),
        'features__text_pipeline__tfidf__use_idf': (True, False),
        'clf__n_estimators': [50, 100, 200],
        'clf__min_samples_split': [2, 3, 4],
        'features__transformer_weights': (
            {'text_pipeline': 1, 'starting_verb': 0.5},
            {'text_pipeline': 0.5, 'starting_verb': 1},
            {'text_pipeline': 0.8, 'starting_verb': 1},
        )
    }

pipeline_cv = GridSearchCV(pipeline, param_grid = parameters)

In [ ]:
%%time
pipeline_cv.fit(X_train, Y_train)
print('Best Parameters:', pipeline_cv.best_params_)

In [ ]:
Y_pred = pipeline_cv.predict(X_test)
    
print(classification_report(Y_test.iloc[:, 1:].values, np.array([x[1:] for x in Y_pred]), target_names = category_names))

### 9. Export your model as a pickle file

In [ ]:
model = simple_pipeline
pickle.dump(model, open(MODEL_PICKLE_FILENAME, 'wb'))

In [ ]:
# Load the model from pickle file

model = pickle.load(open(MODEL_PICKLE_FILENAME, 'rb'))

category_predicted = model.predict(['Storm at sacred heart of jesus'])[0]

In [ ]:
def get_df_from_database(database_filename):
    engine = create_engine('sqlite:///' + database_filename)
    return pd.read_sql_table(TABLE_NAME, engine)

In [ ]:
def get_category_names(database_filename):
    df = get_df_from_database(database_filename)
    return list(df.columns[4:])

print(get_category_names(DATABASE_FILENAME))

In [ ]:
def get_predicted_category_names(category_predicted):
    category_names = get_category_names(DATABASE_FILENAME)
    return [category_names[i] for i in range(len(category_predicted)) if category_predicted[i] == 1]

print(get_predicted_category_names(category_predicted))

In [ ]:
def get_genre_distribution(database_filename):
    df = get_df_from_database(database_filename)
    return df.groupby('genre').count()['message'].to_dict()

print(get_genre_distribution(DATABASE_FILENAME))

In [ ]:
print(list(get_genre_distribution(DATABASE_FILENAME).keys()))

In [ ]:
print(list(get_genre_distribution(DATABASE_FILENAME).values()))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.